This is the second part of the assessment, this time using a python kernel. It incorporates both the linear regressor and the neural network, providing some methods, plus the results and conclusion section of the report. Note that due to the workflow, description of method and results are necesarily intermixed. 

The initial activity for this worksheet was to import the prerequisite data manipulation packages, and then to import the data file from the first part of the assessment that had been uploaded to github after being processed in R.

In [1]:
# Add required packages for first data manipulation steps
import pandas as pd
import numpy as np
# import data file from github
df = pd.read_csv('https://raw.githubusercontent.com/16025833UHI/16025833_DataAnalytics/main/final_col_DF.csv', index_col=0, )

The next step was to verify the correct import of the file.

In [2]:
# checking the first 10 rows to determine if the file has correctly imported
print(df[:10])

#looks ok so far

    Apr  Aug  Dec  Feb  Jan  Jul  Jun  Mar  May  Nov  ...  Tue  Wed  year  \
1     0    0    0    0    0    0    1    0    0    0  ...    0    0  2012   
2     0    0    0    0    0    0    1    0    0    0  ...    0    0  2012   
3     0    0    0    0    0    0    1    0    0    0  ...    1    0  2012   
4     0    0    0    0    0    0    1    0    0    0  ...    0    1  2012   
5     0    0    0    0    0    0    1    0    0    0  ...    0    0  2012   
6     0    0    0    0    0    0    1    0    0    0  ...    0    0  2012   
7     0    0    0    0    0    0    1    0    0    0  ...    0    0  2012   
8     0    0    0    0    0    0    1    0    0    0  ...    0    0  2012   
9     0    0    0    0    0    0    1    0    0    0  ...    0    0  2012   
10    0    0    0    0    0    0    1    0    0    0  ...    1    0  2012   

    day  temp  visib  wdsp  prcp  fog  collisions  
1     1  83.6    9.7   4.1  0.00    0   -0.099605  
2     3  80.3   10.0   3.8  0.00    0    0.20722

Everything appeared to be accurate, with the data structure tranlated without issue from R into python. The next activity was to randomise the dataframe to remove any possible run order that could influence the outcome, and select the correct predictor variables.

In [3]:
# randomising the row order within the dataframe:
rand_df = df.iloc[np.random.permutation(len(df))]

# Selecting all predictor rows related to the linear regressor (note that these are a subset of the columns, other will be used for the DNN regressor)
predictors = rand_df.iloc[:,20:26]

# print out the first 10 rows of predictors to verify that correct variables have been selected.
print(predictors[:10])

      day  temp  visib  wdsp  prcp  fog
1858    2  74.6    6.4   2.3  0.00    0
943     1  18.9    8.8  10.8  0.02    0
2407    3  52.5   10.0   3.4  0.00    0
1051    6  58.0   10.0   4.6  0.00    0
1199    7  55.8   10.0   5.4  0.04    0
949     2  20.5    7.7   4.6  0.00    0
1889    4  49.8   10.0   3.6  0.00    0
1159    1  73.9   10.0   6.3  0.00    0
717     4  79.1   10.0   3.9  0.00    0
2058    6  40.2   10.0   9.3  0.04    0


In [4]:
# print out the shuffled data (first 10 rows)
rand_df[:10]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,...,Tue,Wed,year,day,temp,visib,wdsp,prcp,fog,collisions
1858,0,0,0,0,0,0,0,0,1,0,...,0,0,2017,2,74.6,6.4,2.3,0.00,0,-0.078529
943,0,1,0,0,0,0,0,0,0,0,...,0,0,2015,1,18.9,8.8,10.8,0.02,0,-1.440600
2407,0,0,1,0,0,0,0,0,0,0,...,0,0,2019,3,52.5,10.0,3.4,0.00,0,-0.625047
1051,0,0,0,0,0,1,0,0,0,0,...,0,0,2015,6,58.0,10.0,4.6,0.00,0,-0.441368
1199,0,0,0,0,0,0,0,0,0,0,...,0,0,2015,7,55.8,10.0,5.4,0.04,0,0.945320
949,0,1,0,0,0,0,0,0,0,0,...,0,0,2015,2,20.5,7.7,4.6,0.00,0,-0.798237
1889,0,0,0,0,0,0,0,0,0,0,...,1,0,2017,4,49.8,10.0,3.6,0.00,0,1.059780
1159,0,0,0,0,0,0,0,0,1,0,...,0,0,2015,1,73.9,10.0,6.3,0.00,0,-0.400583
717,0,0,0,0,0,0,1,0,0,0,...,1,0,2014,4,79.1,10.0,3.9,0.00,0,0.626428
2058,0,0,0,1,0,0,0,0,0,0,...,0,0,2018,6,40.2,10.0,9.3,0.04,0,-1.063450


Next, the response variable (collisions) was selected and confirmed to be correct.

In [5]:
# Select the correct row for the response variable
response = rand_df.iloc[:,26]

# print out the first 10 rows of the response variable.
print(response[:10])

1858   -0.078529
943    -1.440600
2407   -0.625047
1051   -0.441368
1199    0.945320
949    -0.798237
1889    1.059780
1159   -0.400583
717     0.626428
2058   -1.063450
Name: collisions, dtype: float64


In [6]:
# Defining a default scale value

SCALE_COLLISIONS = 1.0

The data was split into training and testing subsets in preparation for use with the regressor models, and the predictors and outputs defined. The number of predictors was defined for the linear regressor, but needed to be adjusted later when using the deep neural network model.

In [7]:
#Splitting the data into a test and training set
trainsize = int(len(rand_df['collisions'])*0.8)
testsize = len(rand_df['collisions']) - trainsize

# Defining the number of predictor variables
nppredictors = 6
# Defining the number of response variables
noutputs = 1

With all the preparation steps completed the model could then be run. This involved a number of steps, starting by importing the tensorflow package that would be used for the regression analysis.

In [8]:
# import tensorflow
# %tensorflow_version 1.x
!pip install tensorflow==1.15.2
import tensorflow as tf

# Output version. This should match the specified value provided
print(tf.__version__)

#file management package import
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes the saved model from the previous training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model', ignore_errors=True)

# Setting up the linear regressor

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model using the previously defined predictor and response variables.
estimator.fit(predictors[:trainsize].values, response[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value to the outputs. A multiplication factor of 1 makes no change to the dataset.
predslistscale = preds['scores']*SCALE_COLLISIONS

# Calculate RMSE (root mean square error).
# The RMSE will be minimised by more accurate models.
rmse = np.sqrt(np.mean((response[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));


# Calculate the mean number of collisions from the training set.
avg = np.mean(rand_df['collisions'][:trainsize])

# Calculate the RMSE from the mean of the collisions dataset.
# This allows for a comparison to be made between the efficacy of the linear
# regressor and a very simple estimate.
rmse = np.sqrt(np.mean((rand_df['collisions'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 23 kB/s 
     |████████████████████████████████| 50 kB 2.6 MB/s 
     |████████████████████████████████| 503 kB 34.1 MB/s 
     |████████████████████████████████| 3.8 MB 39.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=d6f24447f314d1057089a5694e8cbbbade339c9df8e96c27fb86b192d97edbae
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting un

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



1.15.2


Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please feed input to tf.data to support dask.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf1d742190>, '_master': '',

starting to train


Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model/model.ckpt.
INFO:tensorflow:loss = 1.0726305, step = 1
INFO:tensorflow:global_step/sec: 811.168
INFO:tensorflow:loss = 0.84682184, step = 101 (0.125 sec)
INFO:tensorflow:global_step/sec: 755.755
INFO:tensorflow:loss = 0.75389504, step = 201 (0.133 sec)
INFO:tensorflow:global_step/sec: 902.63
INFO:tensorflow:loss = 0.7777344, step = 301 (0.115 sec)
INFO:tensorflow:global_step/sec: 799.101
INFO:tensorflow:loss = 0.6777091, step = 401 (0.122 sec)
INFO:tensorflow:global_step/sec: 765.802


LinearRegression has RMSE of 1.0453325385428067
Just using average = 0.005254760292543348 has RMSE of 0.9883410112331039


This indicated that the linear regressor had created an improvement over the average value by reducing the root mean square error (running the model several times gives consistent RMSE reductions in the region of 0.15). This suggested that the model was able to provide a more accurate prediction of collision numbers than the average.

The final part of the process for the model was to make some predictions for the dataset. This was completed by creating a validation dataset (which was a subset of the original randomised dataframe) that had the standardisation removed by multiplying by the standard deviation then adding the mean from the original dataset.

In [9]:
#creating the validation dataset as 5% of the original
valsize = int(len(rand_df['collisions'])*0.05)
#setting the input term and then processing the list
input = predictors.iloc[:valsize]			
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))
#setting predictor values
preds = estimator.predict(x=input.values)
# Rescaling with a mean of 595 and SD of 96.4, based on the mean number of collisions immediately prior to standardisation.
predslistnorm = preds['scores']
predslistscale = preds['scores']*96.4 + 595
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf36ca6e50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

[ 0.5091052  -1.0340208   0.24096501  1.1823919   1.4124062  -0.7101774
  0.45429564  0.16790938  1.1041031   0.79153013  0.5750582   0.22636211
  0.5378773  -0.06419408  0.51148677  1.1791852   1.3303928  -0.05052423
 -0.33434153  1.0319889   0.3625003  -0.391917   -0.04161215  0.24941945
  1.5718737   0.8741853   0.6200421  -0.08101201  0.20774567  0.21555722
  0.36288595 -0.9496809  -0.13231134  0.4861114   1.3372395   0.878144
  0.9777067  -0.02527583  1.4613245  -0.19223928  0.9665296   0.2101928
  1.4555962   1.0201178  -0.26821232  0.6629367   0.16061604  1.7292244
  0.8763933   1.5141051  -0.75680137 -0.5309719   0.54952     0.7862849
  1.6442845   0.45518494  0.9473233   0.41243196 -0.169644    1.1123164
 -0.24597573 -0.22321129  0.02714443  0.61068344  1.393894    0.6492977
  0.34942722  0.78539634  1.1649306   0.56807065  1.4720542  -0.746004
  1.8632791   1.6020503   0.2852497   1.4671876   0.64580894  0.17443836
  0.38109577  0.12939417  0.18941295  0.27712822  1.545855   

As a further exploration, the model was rerun with the removal of the "day" predictor variable. This had a strong relationship with collisions from the data cleaning step, and so with its removal there might have been very limited predicitve power of all the weather variables used. This required minor updates to some of the code used to create the linear regressor originally.

In [10]:
 # Selecting all predictor rows related to the linear regressor (same as previously but removing "day")
predictors = rand_df.iloc[:,21:26]

# print out the first 10 rows of predictors to verify that there are now only 5.
print(predictors[:10])
 
 # Redefining the number of predictor variables
nppredictors = 5
  
# removing the saved model from the previous training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model', ignore_errors=True)

# Setting up the linear regressor

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model using the previously defined predictor and response variables.
estimator.fit(predictors[:trainsize].values, response[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value to the outputs. A multiplication factor of 1 makes no change to the dataset.
predslistscale = preds['scores']*SCALE_COLLISIONS

# Calculate RMSE (root mean square error).
# The RMSE will be minimised by more accurate models.
rmse = np.sqrt(np.mean((response[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

# Calculate the mean number of collisions from the training set.
avg = np.mean(rand_df['collisions'][:trainsize])

# Calculate the RMSE from the mean of the collisions dataset.
# This allows for a comparison to be made between the efficacy of the linear
# regressor and a very simple estimate.
rmse = np.sqrt(np.mean((rand_df['collisions'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf0a4e0450>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Create CheckpointSaverHook.


      temp  visib  wdsp  prcp  fog
1858  74.6    6.4   2.3  0.00    0
943   18.9    8.8  10.8  0.02    0
2407  52.5   10.0   3.4  0.00    0
1051  58.0   10.0   4.6  0.00    0
1199  55.8   10.0   5.4  0.04    0
949   20.5    7.7   4.6  0.00    0
1889  49.8   10.0   3.6  0.00    0
1159  73.9   10.0   6.3  0.00    0
717   79.1   10.0   3.9  0.00    0
2058  40.2   10.0   9.3  0.04    0
starting to train


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model/model.ckpt.
INFO:tensorflow:loss = 1.0726305, step = 1
INFO:tensorflow:global_step/sec: 744.13
INFO:tensorflow:loss = 1.0825284, step = 101 (0.140 sec)
INFO:tensorflow:global_step/sec: 765.756
INFO:tensorflow:loss = 0.9402988, step = 201 (0.132 sec)
INFO:tensorflow:global_step/sec: 554.853
INFO:tensorflow:loss = 1.2817088, step = 301 (0.179 sec)
INFO:tensorflow:global_step/sec: 560.629
INFO:tensorflow:loss = 0.99942917, step = 401 (0.176 sec)
INFO:tensorflow:global_step/sec: 746.477
INFO:tensorflow:loss = 0.9157046, step = 501 (0.136 sec)
INFO:tensorflow:global_step/sec: 753.505
INFO:tensorflow:loss = 1.0223758, step = 601 (0.132 sec)
INFO:tensorflow:global_step/sec: 709.278
INFO:tensorflow:loss = 0.9637076, step = 701 (0.140 sec)
INFO:tensorflow:global_step/sec: 853.821
INFO:tensorflow

LinearRegression has RMSE of 1.1404006115983631
Just using average = 0.005254760292543348 has RMSE of 0.9883410112331039


This led to the RMSE for the mean being almost identical to that generated by the linear regressor, indicating that the weather variables were contributing little to the overall model. Another validation dataset was created for comparison (see next code section). There was visibly a lot less variation in this, which suggested that the model was not applying much correction, as it was lacking strong predictors that would alter the results, and thus had less predictive utility.

In [11]:
#we've already set this, and it doesn't need to change, but worth adding again for visibility
valsize = int(len(rand_df['collisions'])*0.05)
#This also works unchanged because we have simply redefined the "predictors" variable
input = predictors.iloc[:valsize]			
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))
#setting predictor values
preds = estimator.predict(x=input.values)
# Rescaling with a mean of 595 and SD of 96.4, based on the mean number of collisions immediately prior to standardisation.
predslistnorm = preds['scores']
predslistscale = preds['scores']*96.4 + 595
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf0a4e0290>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

[ 0.98806477 -0.2903175   0.3947666   0.516981    0.47632387 -0.2336035
  0.33591726  0.86772096  0.978132    0.1402941   0.46914926  0.6540921
  0.2375826   0.6798513  -0.13458568  1.0973799   0.5272659   0.1111708
  0.09034601  0.9704335   0.7846527   0.04079843  0.11978689  0.69329286
  0.6381576   0.7598653   1.066669    0.61834323 -0.18405229  0.90979457
  0.5155277  -0.1933609   0.2967812   0.3699676   0.93957746  1.0419177
  0.8531356   0.6712706   0.5231483   0.25045475  1.0880506   0.63772535
  1.104847    0.8903754   0.16534868 -0.0044418   0.85314584  0.7994077
  1.0421113   0.6041025  -0.05701071 -0.10281584  0.7061405   0.9894409
  0.9891528   0.69045925  0.8199997   0.08384433 -0.27725175  1.0112463
  0.18907097  0.48147187  0.17735901  0.76730275  0.52324593  0.32297483
  0.26125118  0.4129801   0.79531884  0.72330177  1.0699735  -0.03260282
  0.9644872   0.94149625  0.43886474  0.7941512   0.8030839   0.8643987
  0.26363495  0.55539036  0.8817661   0.17910817  0.8782377

The next stage of the evaluation was to use the deep neural network regressor, which evaluated the one-hot encoded days of the week and months of the year. Most of the settings for the dataset were unchanged, and the variable name "predictors" was retained for simplicity of continuity, but it was assigned different values for use with the new model. The values of "response" were conformed before proceeding further.

In [12]:
# Selecting all predictor rows related to the DNN regressor. Day column has been removed as this would duplicate the one-hot encoded days.
#np.r_ was useful to allow two slicers to be used with iloc.
predictors = rand_df.iloc[:,np.r_[0:19, 21:26]]

# print out the first 10 rows of predictors to verify that correct variables have been selected.
print(predictors[:10])

# Redefining the number of input values (predictors)
nppredictors = 24
# Defining the number of output values (targets). This is not strictly necessary, but helps keep the value visible.
noutputs = 1

#no need to change the response variable, this has already been defined

      Apr  Aug  Dec  Feb  Jan  Jul  Jun  Mar  May  Nov  ...  Sat  Sun  Thu  \
1858    0    0    0    0    0    0    0    0    1    0  ...    1    0    0   
943     0    1    0    0    0    0    0    0    0    0  ...    0    1    0   
2407    0    0    1    0    0    0    0    0    0    0  ...    0    0    0   
1051    0    0    0    0    0    1    0    0    0    0  ...    0    0    1   
1199    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
949     0    1    0    0    0    0    0    0    0    0  ...    1    0    0   
1889    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
1159    0    0    0    0    0    0    0    0    1    0  ...    0    1    0   
717     0    0    0    0    0    0    1    0    0    0  ...    0    0    0   
2058    0    0    0    1    0    0    0    0    0    0  ...    0    0    1   

      Tue  Wed  temp  visib  wdsp  prcp  fog  
1858    0    0  74.6    6.4   2.3  0.00    0  
943     0    0  18.9    8.8  10.8  0.02    0  


The DNN model was then run, and comparisons made between the RMSE values for the mean and linear regressor. 

In [13]:
# removing the saved model from the previous training attempt.
shutil.rmtree('/tmp/DNN_house_regression_trained_model', ignore_errors=True)

# Setting up the DNN regressor
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model using the previously defined predictor and response variables.
estimator.fit(predictors[:trainsize].values, response[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value to the outputs. A multiplication factor of 1 makes no change to the dataset.
predslistscale = preds['scores']*SCALE_COLLISIONS

# Calculate RMSE (root mean square error).
# The RMSE will be minimised by more accurate models.
rmse = np.sqrt(np.mean((response[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean number of collisions from the training set.
avg = np.mean(rand_df['collisions'][:trainsize])

# Calculate the RMSE from the mean of the collisions dataset.
# This allows for a comparison to be made between the efficacy of the DNN and a very simple estimate.
rmse = np.sqrt(np.mean((rand_df['collisions'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf36c5bbd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model', '_session_creation_timeout_secs': 7200}
Instructions for updating:
Please use `layer.__call__` method instead.


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_house_regression_trained_model/model.ckpt.
INFO:tensorflow:loss = 1.479925, step = 1
INFO:tensorflow:global_step/sec: 461.223
INFO:tensorflow:loss = 0.91076326, step = 101 (0.221 sec)
INFO:tensorflow:global_step/sec: 549.408
INFO:tensorflow:loss = 0.62379193, step = 201 (0.183 sec)
INFO:tensorflow:global_step/sec: 389.902
INFO:tensorflow:loss = 0.6110982, step = 301 (0.258 sec)
INFO:tensorflow:global_step/sec: 397.239
INFO:tensorflow:loss = 0.55998814, step = 401 (0.248 sec)
INFO:tensorflow:global_step/sec: 544.034
INFO:tensorflow:loss = 0.518894, step = 501 (0.187 sec)
INFO:tensorflow:global_step/sec: 522.523
INFO:tensorflow:loss = 0.6426079, step = 601 (0.191 sec)
INFO:tensorflow:global_step/sec: 517.542
INFO:tensorflow:loss = 0.62995136, step = 701 (0.196 sec)
INFO:te

DNNRegression has RMSE of 0.7552236644465061
Just using average = 0.005254760292543348 has RMSE of 0.9883410112331039


The DNN regressor generated RMSE values that were very closely comparable to the linear regressor, consistently producing values that were between 0.15 and 0.2 lower than those obtained using the mean value. This again demonstrated that the DNN was providing a useful output. 

The data validation set was re-used to generate values for comparison. Once again, they demonstrated significant variation, aligning closely with the linear regressor.

In [14]:
#we've already set this, and it doesn't need to change, but worth adding again for visibility
valsize = int(len(rand_df['collisions'])*0.05)
#setting the input term. This also works unchanged because we have simply redefined the "predictors" variable
input = predictors.iloc[:valsize]	

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

# Rescaling with a mean of 595 and SD of 96.4, based on the mean number of collisions immediately prior to standardisation.
predslistnorm = preds['scores']
predslistscale = preds['scores']*96.4 + 595
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf0e0748d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_house_regression_trained_model/mo

[-0.02948973 -1.3427128  -0.18028566  0.59215456  0.6424582  -0.6360307
  0.4279219  -1.0040275   0.32339174 -0.1516688  -0.017849   -0.08833429
 -0.49370408 -0.7040498   1.0390565   0.34689367  2.3087125   1.1713661
 -0.43906075  0.06637898 -0.5409424  -0.7368579   1.1809335  -0.48112133
  0.7157526   0.79345965  0.00661242 -0.84199476  0.26990765 -0.87123686
 -0.11561808 -1.4766548  -0.54754496 -0.46046275  0.21454269  1.2885704
  0.22643924 -0.74725425  0.87181437 -0.15799862  1.3243544  -0.07418022
  0.3314371   0.04268405 -0.7840363   0.17103955 -0.9762785   1.1603217
  0.34468997  1.0844274  -1.558015   -1.0018145  -0.23248091  1.0539764
  0.41920382  0.7777729   0.4621212  -0.47581643  1.0306216   0.36762637
 -0.50865984 -1.2977378   0.33388364 -0.06245589  2.0857997   0.00275281
  0.0412311  -0.08496071  0.7174747  -0.1593669   0.25618827 -1.4451538
  1.327472    0.13873586 -0.10176203  0.20415306 -0.05053201 -0.8032986
 -0.38887158 -0.4849695  -0.9390375  -0.17017832  0.116156

The model was then rerun using some alternative neural network model settings. Specifically this meant increasing the number of nodes and layers within the network. The values were increased from 3 layers (20, 18, 14 nodes) to 5 layers (24, 21, 18, 15, 12 nodes). There were no alterations made to the input or output variables. The number of model training steps was also increased from 10,000 to 15,000.

In [15]:
# No changes to variables used.

# removing the saved model from the previous training attempt.
shutil.rmtree('/tmp/DNN_house_regression_trained_model', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[24,21,18,15,12], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model using the previously defined predictor and response variables.
estimator.fit(predictors[:trainsize].values, response[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=15000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value to the outputs. A multiplication factor of 1 makes no change to the dataset.
predslistscale = preds['scores']*SCALE_COLLISIONS

# Calculate RMSE (root mean square error).
# The RMSE will be minimised by more accurate models.
rmse = np.sqrt(np.mean((response[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));

# Calculate the mean number of collisions from the training set.
avg = np.mean(rand_df['collisions'][:trainsize])

# Calculate the RMSE from the mean of the collisions dataset.
# This allows for a comparison to be made between the efficacy of the DNN and a very simple estimate.
rmse = np.sqrt(np.mean((rand_df['collisions'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf0e130190>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_house_regression_trained_model/model.ckpt.
INFO:tensorflow:loss = 4.7972536, step = 1
INFO:tensorflow:global_step/sec: 380.329
INFO:tensorflow:loss = 0.88315, step = 101 (0.268 sec)
INFO:tensorflow:global_step/sec: 536.613
INFO:tensorflow:loss = 0.61765385, step = 201 (0.186 sec)
INFO:tensorflow:global_step/sec: 496.854
INFO:tensorflow:loss = 0.60583496, step = 301 (0.202 sec)
INFO:tensorflow:global_step/sec: 495.449
INFO:tensorflow:loss = 0.49713734, step = 401 (0.201 sec)
INFO:tensorflow:global_step/sec: 479.682
INFO:tensorflow:loss = 0.47910196, step = 501 (0.208 sec)
INFO:tensorflow:global_step/sec: 532.546
INFO:tensorflow:loss = 0.5955589, step = 601 (0.188 sec)
INFO:tensorflow:global_step/sec: 508.963
INFO:tensorflow:loss = 0.594618, step = 701 (0.195 sec)
INFO:ten

DNNRegression has RMSE of 0.8018945667529916
Just using average = 0.005254760292543348 has RMSE of 0.9883410112331039


The RMSEs produced from this altered model varied, but were typically again between 0.15 and 0.2 lower than the mean. There was a risk of overfitting the model for the data at this stage, and the potential gains were small. A final comparison of the validation data set was completed, which again demonstrated values ranging from approximately 350 to 750, which is very similar to the full variation of data observed from the raw collisions dataset before standardisation.

In [16]:
#we've already set this, and it doesn't need to change, but worth adding again for visibility
valsize = int(len(rand_df['collisions'])*0.05)
#setting the input term. This also works unchanged because we have simply redefined the "predictors" variable
input = predictors.iloc[:valsize]	

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[24,21,18,15,12], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

# Rescaling with a mean of 595 and SD of 96.4, based on the mean number of collisions immediately prior to standardisation.
predslistnorm = preds['scores']
predslistscale = preds['scores']*96.4 + 595
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf0e116c90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_house_regression_trained_model/mo

[-0.57966185 -1.6316152  -0.21459748  0.454942    0.7701213  -1.0531939
  0.35320175 -0.9655986   0.41390365 -0.3704991  -0.11127317 -0.4492696
  0.31652206 -0.6326879   0.8810074   0.51373065  2.256728    0.9220033
 -0.7527927   0.00421345 -0.54708266 -1.2344701   0.88652414 -0.62653214
  0.97810465  0.49457812 -0.6014431  -0.8933561   0.23044485 -1.1435741
 -0.33813497 -1.6500266  -0.6333233  -0.45041487  0.11912388  1.380003
  0.10978341 -0.62575114  1.2309742   0.4678443   1.8344367  -0.4492696
 -0.0672535  -0.11028218 -1.0318371   0.46503776 -1.1469896   1.8167763
  0.28807986  1.0981057  -1.7006056  -0.18676108 -0.01414067  0.9308931
  0.4279644   0.9990683   0.3403206  -0.406751   -0.0089314   0.23550886
 -0.6814385  -1.5061362  -1.4389253  -0.4492696   1.3032439   0.04798964
 -0.5175773   0.05482095  0.71910745 -0.34180957  0.4274605  -1.6379426
  1.8336408  -0.08078942 -0.34348717 -0.05528671 -0.26696807 -0.93232614
 -0.4492696  -0.50077635 -1.1246266  -0.6815827  -0.06197098 

Conclusions:

The collisons data was successfully tidied and prepared for the linear and DNN regressors, with the high degree of initial variability removed. This led to strong RMSE reductions vs average values, indicating that the models were able to work effectively on the data provided to produce consistent predictions. There were limited differences observed between the linear and DNN regressors, suggesting that the predictor variables available were not sufficient for the models to create more accurate predictions. On a similar note, the removal of the weather varaible from one run of the linear regressor indicated that the day of the week predictor was providing almost all of the models' predictive power. The implication of this observation was that the weather factors were generally very weak in predicting the number of collisions, in agreement with the low correlation values recorded during initial evaluation of the data. This demonstrates that with a large dataset there is a risk of generating statistically significant corellations when the practical impact of such relationships is very small.

A weakness of this study was using the same dataset for the training, testing and validation. A better option would have been to generate a dataset from a different year for evaluation, to determine if the model would still perform well. Creating the validation data from the teaching set only demonstrates the strength of the model with the data already encoutnered, and so general utility cannot easily be extrapolated. However the process of generating an additional dataset that correctly matches the format requirements after data cleaning would have been time consuming. The best option would have been to create the validation data set at the same time as the main data set, to make the workflow more streamlined.

A second opportunity with workflow would have been to return the outputs of the validation process to R for further visualisation. Within the scope of this report this would have been a challenge, as the worksheet sequencing would have become complex, but plotting the regressor outputs against (for example) days of the week would have an improvement in demonstrating how successfully the models were performing.

Finally, if a measure of total traffic activity were available then the collisions data could be expressed relative to the number of vehicle journeys. This would be likely to result in a flat profile, which could then potentially be used as a cleaner background signal to detect subtle differences in weather effects. Overall the best recommendation for emergency service presence on the roads in this report was be to follow the days of the week collision pattern identified: lowest presence on Sunday, more on Saturday, then further increases from Monday to Friday.


References:

The packages used for data processing in this section of the report were Pandas, NumPy and Tensorflow. The references for these (together with a citation for Python) are as follows, in order of use:

Van Rossum, G. & Drake Jr, F.L., 1995. Python reference manual, Centrum voor Wiskunde en Informatica Amsterdam.

McKinney, W. & others, 2010. Data structures for statistical computing in python. In Proceedings of the 9th Python in Science Conference. pp. 51–56.

Harris, C.R. et al., 2020. Array programming with NumPy. Nature, 585, pp.357–362.

Abadi, Mart&#x27;in et al., 2016. Tensorflow: A system for large-scale machine learning. In 12th USENIX Symposium on Operating Systems Design and Implementation (OSDI 16). pp. 265–283.

Note that the Shutil package is standar module of Python and is therefore not cited.